In [3]:
import pandas as pd
import numpy as np

In [14]:
# Importer le fichier population et csp (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_RP_POPULATION_COMP "Télécharger la totalité du jeu de données")
df = pd.read_csv(r'DS_RP_POPULATION_COMP_2022_data.csv', sep=';')

/tmp/ipykernel_47872/4169547382.py:2: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'DS_RP_POPULATION_COMP_2022_data.csv', sep=';')


In [36]:
# Sélection des communes uniquement, de l'année 2022, du sexe _T et de la tranche d'âge 'Y_GE15'
df1 = df[df['GEO_OBJECT'] == 'COM']
df1 = df1[df1['TIME_PERIOD'] == 2022]
df1 = df1[df1['SEX'] == '_T']
df1 = df1[df1['AGE'] == 'Y_GE15']

In [38]:
# Drop des colonnes inutiles
df1 = df1.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'SEX', 'RP_MEASURE', 'AGE'])

In [47]:
# Arrondis les valeurs de OBS à l'entier supérieur
df1["OBS_VALUE"] = np.ceil(df1["OBS_VALUE"])

In [48]:
df1.head()

,GEO,PCS,OBS_VALUE
731762,68157,6,160
731794,68340,3,16
731797,68329,6,144
731810,68167,3,21
731816,69141,3,694


In [49]:
# Exporter df1 en CSV
df1.to_csv('population_communes_2022_csp.csv', index=False)

In [4]:
# Importer le fichier population municipale totale (population de référence) (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_POPULATIONS_REFERENCE "Télécharger la totalité du jeu de données")
df2 = pd.read_csv(r'DS_POPULATIONS_REFERENCE_data.csv', sep=';')

In [5]:
# Sélectionner les communes, l'année 2022 et la population municipale (PMUN)
df2 = df2[df2['TIME_PERIOD'] == 2022]
df2 = df2[df2['GEO_OBJECT'] == 'COM']
df2 = df2[df2['POPREF_MEASURE'] == 'PMUN']

In [6]:
# Supprimer les colonnes GEO_OBJECT, TIME_PERIOD, POPREF_MEASURE et FREQ
df2 = df2.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'POPREF_MEASURE', 'FREQ'])

In [7]:
df2.head()

,GEO,OBS_VALUE
4858,03066,321
4872,07049,60
4874,04101,139
4875,04067,59
4889,12268,424


In [ ]:

def calculer_conseillers_municipaux(fichier_repartition):
   """
   Calcule le nombre de conseillers municipaux pour chaque commune à partir du fichier de population municipale (df2) et du fichier "conseillers.csv" qui fournit la répartition selon les fourchettes de population.
   
   Parameter:
   -----------
   fichier_repartition : str
       Chemin vers le fichier CSV avec la répartition population/conseillers
       
   Returns:
   --------
   pandas.DataFrame
       DataFrame avec les communes et leur nombre de conseillers
   """
   
   # Charger le fichier
   repartition_df = pd.read_csv(fichier_repartition)
   
   # Charger le dataframe population
   communes_df = df2
   
   # Fonction pour déterminer le nombre de conseillers
   def get_nb_conseillers(population):
       for _, row in repartition_df.iterrows():
           if row['population_min'] <= population <= row['population_max']:
               return row['nombre_de_conseillers']
       return None
   
   # Calculer le nombre de conseillers pour chaque commune
   communes_df['nombre_conseillers'] = communes_df['OBS_VALUE'].apply(get_nb_conseillers)
   
   return communes_df

In [ ]:
# Lancer la fonction
df3 = calculer_conseillers_municipaux('./data/conseillers.csv')

In [12]:
df3.head()

,GEO,OBS_VALUE,nombre_conseillers
4858,03066,321,11
4872,07049,60,7
4874,04101,139,11
4875,04067,59,7
4889,12268,424,11


In [13]:
# Exporter df3 en CSV
df3.to_csv('population_municipale_2022_et_conseillers.csv', index=False)